In [2]:
import os, gc, time
USER = os.getenv('USER')
import numpy as np
import pandas as pd
import xarray as xr
import dask

dask.config.set(**{'array.slicing.split_large_chunks': False})

from dscim import ProWaiter
import itertools
from functools import reduce
from pathlib import Path

In [3]:
discount_types = ["euler_ramsey","constant"]
menu_options = ["risk_aversion", "adding_up"] 
# pulse_years=[2020]
pulse_years = range(2020, 2081, 10)
weitzman_values = [x / 10.0 for x in range(1, 11, 1)] + [0.25, 0.01, 0.001, 0.0001]
eta_rhos = { 
    2.0:0.0,
    1.016010255 : 9.149608e-05,
    1.244459066 : 0.00197263997,
    1.421158116 : 0.00461878399,
    1.567899395 : 0.00770271076
}

# options
factors=True
marginal_damages=True
global_cons=False

# we're doing expost masking with rff_expost_masking.ipynb inside this same directory
mask_list = [
#     'climate',
#     'gdppc',
#     'emissions',
#     'gdppc_emissions',
#     'gdppc_emissions_climate',
    'unmasked',
    # 'gmst'

]

quantile_list = [
    "None"
#     [0.001, 0.999],
#     [0.005, 0.995],
    # [0.01, 0.99],
    # [0.05, 0.95],
    # [0,1],
]

sectors = ['mortality', 'mortality_vsl_popavg', 'mortality_vsl_ir']
# sectors = ['CAMEL']

combos = [(s, p, d, m, k, q, e) 
          for p in pulse_years
          for s in sectors 
          for d in discount_types 
          for m in menu_options
          for k in mask_list
          for q in quantile_list
          for e in eta_rhos
         ]


# run all

In [4]:
n_combos = len(combos)

while n_combos > 0: 
    combo = combos[0]
    print("=========================================================")
    print(combo)
    sector = combo[0]
    pulse_year = combo[1]
    discount_type = combo[2]
    menu_option = combo[3]
    mask = combo[4]
    quantile = combo[5]
    eta = combo[6]
    rho = eta_rhos[eta]

    if mask == 'unmasked':
        mask_name = None
        mask_path = None
    elif mask == 'gmst':
        mask_name =  f'q{quantile[0]}_q{quantile[1]}'
        mask_path = None
    else:
        mask_name = f'q{quantile[0]}_q{quantile[1]}'
        mask_path = f"/shares/gcp/integration/rff/climate/masks/CO2_Fossil/{mask}_based_masks.nc4"

    save_path = f'/mnt/CIL_integration/rff2/{sector}/{pulse_year}/{mask}_{mask_name}'
    
    w = ProWaiter(path_to_config=f'/home/{USER}/repos/integration/configs/rff2_config_all_gases.yaml')
    kwargs = {'discounting_type' : discount_type,
              'sector': sector,
              'damage_function_path' : f"/mnt/CIL_integration/damage_function_library/damage_function_library_rff2/{sector}",
              'save_path' : save_path,
              'save_files' : ['uncollapsed_sccs'],
              'weitzman_parameter' : weitzman_values,
              'pulse_year' : pulse_year,
              'ecs_mask_path' :  mask_path,
              'ecs_mask_name' :  mask_name,
              'eta' : eta,
              'rho' : rho,
             }
    
    menu_item = w.menu_factory(menu_key=menu_option,
                                           sector=sector,
                                           kwargs=kwargs
                                          )
    # menu_item.order_plate('scc')
    
    if global_cons == True:
        menu_item.global_consumption_no_pulse.to_netcdf(
            f"{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_global_consumption_no_pulse.nc4")
        menu_item.global_consumption.to_netcdf(
            f"{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_global_consumption.nc4")


    if marginal_damages==True:
        md = (
            (menu_item.global_consumption_no_pulse - menu_item.global_consumption_pulse) 
            * menu_item.climate.conversion
        ).rename('marginal_damages').to_dataset()
        
        for var in md.variables:
            md[var].encoding.clear()
            
        md.chunk(
        {"discount_type":1,
         "weitzman_parameter":14,
         "runid":10000,
         "gas":1,
         "year":10}
        ).to_zarr(
            f'{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_uncollapsed_marginal_damages.zarr',
            consolidated = True,
            mode = "w"
        )
    if factors == True:
        
        f = menu_item.calculate_discount_factors(
            menu_item.global_consumption_no_pulse / menu_item.pop
        ).to_dataset(name = "discount_factor")
        
        for var in f.variables:
            f[var].encoding.clear()
        
        f.chunk(
        {"discount_type":1,
         "weitzman_parameter":14,
         "runid":10000,
         "gas":1,
         "region":1,
         "year":10}
        ).to_zarr(
            f'{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_uncollapsed_discount_factors.zarr',
            consolidated = True,
            mode = "w"
        )
        print("done saving discount factor")
    
    menu_item.order_plate('scc')
    combos.remove(combo)
    n_combos = len(combos)
    print(f"remaining combos: {n_combos}")

('mortality', 2020, 'euler_ramsey', 'equity', 'unmasked', 'None', 2.0)
Skipped saving damage_function_coefficients.
Subsetting on {} keys.
Global consumption found up to 2300.
Skipped saving global_consumption.
Skipped saving damage_function_coefficients.
Subsetting on {} keys.


/home/liruixue/repos/integration/dscim/menu/main_recipe.py:1174: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  rhos = xr.ufuncs.expm1(xr.DataArray(self.rho, coords=[cons_pc.year]))
/home/liruixue/miniconda3/envs/dscim/lib/python3.8/site-packages/xarray/core/dataarray.py:3061: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  da = self.__array_wrap__(f(self.variable.data, *args, **kwargs))


done saving discount factor

 Executing 
        Running equity
        sector: mortality
        discounting: euler_ramsey
        eta: 2.0
        rho: 0.0
        
Processing damage functions ...
Skipped saving damage_function_coefficients.
Processing SCC calculation ...
Saving /mnt/CIL_integration/rff2/mortality/2020/unmasked_None/equity_euler_ramsey_eta2.0_rho0.0_uncollapsed_sccs.nc4


/home/liruixue/repos/integration/dscim/menu/main_recipe.py:1174: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  rhos = xr.ufuncs.expm1(xr.DataArray(self.rho, coords=[cons_pc.year]))
/home/liruixue/miniconda3/envs/dscim/lib/python3.8/site-packages/xarray/core/dataarray.py:3061: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  da = self.__array_wrap__(f(self.variable.data, *args, **kwargs))


Results available: /mnt/CIL_integration/rff2/mortality/2020/unmasked_None
remaining combos: 629
('mortality', 2020, 'euler_ramsey', 'equity', 'unmasked', 'None', 1.016010255)
Skipped saving damage_function_coefficients.
Subsetting on {} keys.
Global consumption found up to 2300.
Skipped saving global_consumption.
Skipped saving damage_function_coefficients.


...

/home/liruixue/repos/integration/dscim/menu/main_recipe.py:1174: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  rhos = xr.ufuncs.expm1(xr.DataArray(self.rho, coords=[cons_pc.year]))
/home/liruixue/miniconda3/envs/dscim/lib/python3.8/site-packages/xarray/core/dataarray.py:3061: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  da = self.__array_wrap__(f(self.variable.data, *args, **kwargs))


done saving discount factor

 Executing 
        Running equity
        sector: mortality
        discounting: euler_ramsey
        eta: 1.567899395
        rho: 0.00770271076
        
Processing damage functions ...
Skipped saving damage_function_coefficients.
Processing SCC calculation ...
Saving /mnt/CIL_integration/rff2/mortality/2020/unmasked_None/equity_euler_ramsey_eta1.567899395_rho0.00770271076_uncollapsed_sccs.nc4


/home/liruixue/repos/integration/dscim/menu/main_recipe.py:1174: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  rhos = xr.ufuncs.expm1(xr.DataArray(self.rho, coords=[cons_pc.year]))
/home/liruixue/miniconda3/envs/dscim/lib/python3.8/site-packages/xarray/core/dataarray.py:3061: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  da = self.__array_wrap__(f(self.variable.data, *args, **kwargs))


Results available: /mnt/CIL_integration/rff2/mortality/2020/unmasked_None
remaining combos: 625
('mortality', 2020, 'euler_ramsey', 'risk_aversion', 'unmasked', 'None', 2.0)
Skipped saving damage_function_coefficients.
Subsetting on {} keys.
Global consumption found up to 2300.
Skipped saving global_consumption.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

